In [25]:
from transformers import BertTokenizer, BertForSequenceClassification
from transformers import Trainer, TrainingArguments
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import f1_score, classification_report
import torch
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np
from transformers import AdamW
from tqdm import tqdm

In [26]:
language="tr"
modelname="mBert"
input = "gpt4"
test = "Dodiom"
filtered= False

In [27]:
if input == "gpt4":
  gpt4_flag=True
else:
  gpt4_flag=False

In [28]:
# JP encoding='euc-jp' other encoding normal CHECK ALWAYS DATASETS
df_train = pd.read_csv(f'train_gpt4_tr_enhanced_prompting.csv')
df_eval = pd.read_csv(f'eval_gpt4_tr_enhanced_prompting.csv')
df_test = pd.read_csv(f'dodiom_tr_test.csv')

In [29]:
def get_category_mapping(language):
    if language in ['en', 'jp']:
        category_mapping = {
            'figurative': 'idiom',
            'literal': 'nonidiom'
        }
    elif language == 'tr':
        category_mapping = {
            'mecaz': 'idiom',
            'gerçek': 'nonidiom'
        }
    elif language == 'it':
        category_mapping = {
            'figurato': 'idiom',
            'letterale': 'nonidiom'
        }
    else:
        raise ValueError(f"Language '{language}' is not supported.")

    return category_mapping

In [30]:
category_mapping = get_category_mapping(language)
print(category_mapping)

{'mecaz': 'idiom', 'gerçek': 'nonidiom'}


In [31]:
if gpt4_flag or language in ['en', 'jp']:
  df_train['category'] = df_train['category'].map(category_mapping)
df_train

,submission,category,idiom,type
0,Şirketteki rekabetçi ortamda başa geçmek herke...,idiom,Başa geçmek,Enhanced-prompting
1,Yeni kurulan bu sivil toplum kuruluşunda proje...,idiom,Yer almak,Enhanced-prompting
2,"Araştırma raporunun sonuçları, beklenmedik ver...",idiom,Ortaya çıkmak,Enhanced-prompting
3,Bu tatilde arkadaşlarla bir hafta daha kalabil...,idiom,El açmak,Enhanced-prompting
4,Deprem anında nasıl sıkı durabiliriz?,idiom,Sıkı durmak,Enhanced-prompting
...,...,...,...,...
2299,"Yüksek kayanın üzerinde duran maceracı, etrafı...",nonidiom,Üzerinde durmak,Enhanced-prompting
2300,"Köpeğim bahçede koşarken, ıslak çimler üzerind...",nonidiom,İz bırakmak,Enhanced-prompting
2301,Kaynamış su şişenin plastik içini yavaşça erit...,nonidiom,İçi erimek,Enhanced-prompting
2302,"Herkes için zaman sınırlıyken, onu doğru değer...",nonidiom,Zaman kazanmak,Enhanced-prompting


In [32]:
if gpt4_flag or language in ['en', 'jp']:
  df_eval['category'] = df_eval['category'].map(category_mapping)
df_eval

,submission,category,idiom,type
0,Kriz anında gereken liderlik özelliklerini gös...,idiom,Başa geçmek,Enhanced-prompting
1,"Çalışma grupları, genellikle ortaya çıkan soru...",idiom,Parmak basmak,Enhanced-prompting
2,"Tartışma sırasında, Anna'nın sürekli aynı nokt...",idiom,Üzerinde durmak,Enhanced-prompting
3,Yarışta istenilen performansı sergileyemediği ...,idiom,Başa geçmek,Enhanced-prompting
4,İlişkide sürekli problem yaşayıp da sıkı durmu...,idiom,Sıkı durmak,Enhanced-prompting
...,...,...,...,...
571,Okul bahçesinde çocuklar oyun oynayacak yer bu...,nonidiom,Yer almak,Enhanced-prompting
572,Bahçede toplanan meyveleri sepete eliyle koyma...,nonidiom,El koymak,Enhanced-prompting
573,"Bahçıvan, çiçek dikimi için sabah erkenden kol...",nonidiom,Kolları sıvamak,Enhanced-prompting
574,Çocuk parkta scooterını ileri sürerken yüksek ...,nonidiom,İleri sürmek,Enhanced-prompting


In [33]:
if language in ['en', 'jp']:
  df_test['category'] = df_test['category'].map(category_mapping)

In [34]:
df_test

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words
0,adım atmak,"Sen bu adama adım mı attın ki, o sana yakınlaş...",idiom,0.75,3,0,1,"[3, 5]","['adım', 'attın']","['se', 'bu', 'ada', 'adım', 'mı', 'atmak', 'ki...","['Sen', 'bu', 'adama', 'adım', 'mı', 'attın', ..."
1,adım atmak,Bütün zorluklara rağmen yeni bir işe adım attı.,idiom,1.00,3,0,0,"[6, 7]","['adım', 'attı']","['bütün', 'zor', 'rağmen', 'yen', 'bir', 'işem...","['Bütün', 'zorluklara', 'rağmen', 'yeni', 'bir..."
2,adım atmak,Adım attığı bu hayatın başına ne gibi olaylar ...,idiom,0.75,3,0,1,"[0, 1]","['Adım', 'attığı']","['adım', 'atmak', 'bu', 'hayat', 'baş', 'ne', ...","['Adım', 'attığı', 'bu', 'hayatın', 'başına', ..."
3,adım atmak,"Bölüm başkanı, akademik dünyaya adım atan genç...",idiom,0.50,1,0,1,"[5, 6]","['adım', 'atan']","['bölüm', 'başka', ',', 'akademik', 'Dünya', '...","['Bölüm', 'başkanı', ',', 'akademik', 'dünyaya..."
4,adım atmak,Yeni bir deneyime adım atıyorum.,idiom,0.75,3,0,1,"[3, 4]","['adım', 'atıyorum']","['yen', 'bir', 'deneyim', 'adım', 'atmak', '.']","['Yeni', 'bir', 'deneyime', 'adım', 'atıyorum'..."
...,...,...,...,...,...,...,...,...,...,...,...
2079,üzerinde durmak,Bu ipin üzerinde duramıyorum.,nonidiom,1.00,5,0,0,"[2, 3]","['üzerinde', 'duramıyorum']","['bu', 'ip', 'Üzer', 'durmak', '.']","['Bu', 'ipin', 'üzerinde', 'duramıyorum', '.']"
2080,üzerinde durmak,İpin üzerinde tek ayağıyla duruyordu.,nonidiom,1.00,5,0,0,"[1, 4]","['üzerinde', 'duruyordu']","['ip', 'Üzer', 'tek', 'Ayağ', 'durmak', '.']","['İpin', 'üzerinde', 'tek', 'ayağıyla', 'duruy..."
2081,üzerinde durmak,Bilgisayarın üzerinde kitapları duruyor,nonidiom,1.00,4,0,0,"[1, 3]","['üzerinde', 'duruyor']","['bilgisayar', 'Üzer', 'kitap', 'durmak']","['Bilgisayarın', 'üzerinde', 'kitapları', 'dur..."
2082,üzerinde durmak,Üzerinde çilekler duran tartı bana uzattı.,nonidiom,1.00,1,0,0,"[0, 2]","['Üzerinde', 'duran']","['Üzer', 'Çilek', 'durmak', 'tartı', 'ban', 'u...","['Üzerinde', 'çilekler', 'duran', 'tartı', 'ba..."


In [35]:
if filtered == True:
  df_train = df_train.groupby(['idiom', 'category']).apply(lambda x: x.sample(n=32, replace=False)).reset_index(drop=True)
  df_eval = df_eval.groupby(['idiom', 'category']).apply(lambda x: x.sample(n=8, replace=False)).reset_index(drop=True)

In [36]:
df_train['category'] = df_train['category'].str.lower()
df_eval['category'] = df_eval['category'].str.lower()
df_test['category'] = df_test['category'].str.lower()

In [37]:
df_train.head()

,submission,category,idiom,type
0,Şirketteki rekabetçi ortamda başa geçmek herke...,idiom,Başa geçmek,Enhanced-prompting
1,Yeni kurulan bu sivil toplum kuruluşunda proje...,idiom,Yer almak,Enhanced-prompting
2,"Araştırma raporunun sonuçları, beklenmedik ver...",idiom,Ortaya çıkmak,Enhanced-prompting
3,Bu tatilde arkadaşlarla bir hafta daha kalabil...,idiom,El açmak,Enhanced-prompting
4,Deprem anında nasıl sıkı durabiliriz?,idiom,Sıkı durmak,Enhanced-prompting


In [38]:
df_eval.head()

,submission,category,idiom,type
0,Kriz anında gereken liderlik özelliklerini gös...,idiom,Başa geçmek,Enhanced-prompting
1,"Çalışma grupları, genellikle ortaya çıkan soru...",idiom,Parmak basmak,Enhanced-prompting
2,"Tartışma sırasında, Anna'nın sürekli aynı nokt...",idiom,Üzerinde durmak,Enhanced-prompting
3,Yarışta istenilen performansı sergileyemediği ...,idiom,Başa geçmek,Enhanced-prompting
4,İlişkide sürekli problem yaşayıp da sıkı durmu...,idiom,Sıkı durmak,Enhanced-prompting


In [39]:
df_test.head()

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words
0,adım atmak,"Sen bu adama adım mı attın ki, o sana yakınlaş...",idiom,0.75,3,0,1,"[3, 5]","['adım', 'attın']","['se', 'bu', 'ada', 'adım', 'mı', 'atmak', 'ki...","['Sen', 'bu', 'adama', 'adım', 'mı', 'attın', ..."
1,adım atmak,Bütün zorluklara rağmen yeni bir işe adım attı.,idiom,1.00,3,0,0,"[6, 7]","['adım', 'attı']","['bütün', 'zor', 'rağmen', 'yen', 'bir', 'işem...","['Bütün', 'zorluklara', 'rağmen', 'yeni', 'bir..."
2,adım atmak,Adım attığı bu hayatın başına ne gibi olaylar ...,idiom,0.75,3,0,1,"[0, 1]","['Adım', 'attığı']","['adım', 'atmak', 'bu', 'hayat', 'baş', 'ne', ...","['Adım', 'attığı', 'bu', 'hayatın', 'başına', ..."
3,adım atmak,"Bölüm başkanı, akademik dünyaya adım atan genç...",idiom,0.50,1,0,1,"[5, 6]","['adım', 'atan']","['bölüm', 'başka', ',', 'akademik', 'Dünya', '...","['Bölüm', 'başkanı', ',', 'akademik', 'dünyaya..."
4,adım atmak,Yeni bir deneyime adım atıyorum.,idiom,0.75,3,0,1,"[3, 4]","['adım', 'atıyorum']","['yen', 'bir', 'deneyim', 'adım', 'atmak', '.']","['Yeni', 'bir', 'deneyime', 'adım', 'atıyorum'..."


In [40]:
# Encode the 'category' column
label_encoder = LabelEncoder()
df_train['category_encoded'] = label_encoder.fit_transform(df_train['category'])
df_eval['category_encoded'] = label_encoder.transform(df_eval['category'])
df_test['category_encoded'] = label_encoder.transform(df_test['category'])

In [41]:
df_train

,submission,category,idiom,type,category_encoded
0,Şirketteki rekabetçi ortamda başa geçmek herke...,idiom,Başa geçmek,Enhanced-prompting,0
1,Yeni kurulan bu sivil toplum kuruluşunda proje...,idiom,Yer almak,Enhanced-prompting,0
2,"Araştırma raporunun sonuçları, beklenmedik ver...",idiom,Ortaya çıkmak,Enhanced-prompting,0
3,Bu tatilde arkadaşlarla bir hafta daha kalabil...,idiom,El açmak,Enhanced-prompting,0
4,Deprem anında nasıl sıkı durabiliriz?,idiom,Sıkı durmak,Enhanced-prompting,0
...,...,...,...,...,...
2299,"Yüksek kayanın üzerinde duran maceracı, etrafı...",nonidiom,Üzerinde durmak,Enhanced-prompting,1
2300,"Köpeğim bahçede koşarken, ıslak çimler üzerind...",nonidiom,İz bırakmak,Enhanced-prompting,1
2301,Kaynamış su şişenin plastik içini yavaşça erit...,nonidiom,İçi erimek,Enhanced-prompting,1
2302,"Herkes için zaman sınırlıyken, onu doğru değer...",nonidiom,Zaman kazanmak,Enhanced-prompting,1


In [42]:
df_eval

,submission,category,idiom,type,category_encoded
0,Kriz anında gereken liderlik özelliklerini gös...,idiom,Başa geçmek,Enhanced-prompting,0
1,"Çalışma grupları, genellikle ortaya çıkan soru...",idiom,Parmak basmak,Enhanced-prompting,0
2,"Tartışma sırasında, Anna'nın sürekli aynı nokt...",idiom,Üzerinde durmak,Enhanced-prompting,0
3,Yarışta istenilen performansı sergileyemediği ...,idiom,Başa geçmek,Enhanced-prompting,0
4,İlişkide sürekli problem yaşayıp da sıkı durmu...,idiom,Sıkı durmak,Enhanced-prompting,0
...,...,...,...,...,...
571,Okul bahçesinde çocuklar oyun oynayacak yer bu...,nonidiom,Yer almak,Enhanced-prompting,1
572,Bahçede toplanan meyveleri sepete eliyle koyma...,nonidiom,El koymak,Enhanced-prompting,1
573,"Bahçıvan, çiçek dikimi için sabah erkenden kol...",nonidiom,Kolları sıvamak,Enhanced-prompting,1
574,Çocuk parkta scooterını ileri sürerken yüksek ...,nonidiom,İleri sürmek,Enhanced-prompting,1


In [43]:
df_test

,idiom,submission,category,rating,likes,dislikes,reports,idiom_indices,idiom_words,lemmas,words,category_encoded
0,adım atmak,"Sen bu adama adım mı attın ki, o sana yakınlaş...",idiom,0.75,3,0,1,"[3, 5]","['adım', 'attın']","['se', 'bu', 'ada', 'adım', 'mı', 'atmak', 'ki...","['Sen', 'bu', 'adama', 'adım', 'mı', 'attın', ...",0
1,adım atmak,Bütün zorluklara rağmen yeni bir işe adım attı.,idiom,1.00,3,0,0,"[6, 7]","['adım', 'attı']","['bütün', 'zor', 'rağmen', 'yen', 'bir', 'işem...","['Bütün', 'zorluklara', 'rağmen', 'yeni', 'bir...",0
2,adım atmak,Adım attığı bu hayatın başına ne gibi olaylar ...,idiom,0.75,3,0,1,"[0, 1]","['Adım', 'attığı']","['adım', 'atmak', 'bu', 'hayat', 'baş', 'ne', ...","['Adım', 'attığı', 'bu', 'hayatın', 'başına', ...",0
3,adım atmak,"Bölüm başkanı, akademik dünyaya adım atan genç...",idiom,0.50,1,0,1,"[5, 6]","['adım', 'atan']","['bölüm', 'başka', ',', 'akademik', 'Dünya', '...","['Bölüm', 'başkanı', ',', 'akademik', 'dünyaya...",0
4,adım atmak,Yeni bir deneyime adım atıyorum.,idiom,0.75,3,0,1,"[3, 4]","['adım', 'atıyorum']","['yen', 'bir', 'deneyim', 'adım', 'atmak', '.']","['Yeni', 'bir', 'deneyime', 'adım', 'atıyorum'...",0
...,...,...,...,...,...,...,...,...,...,...,...,...
2079,üzerinde durmak,Bu ipin üzerinde duramıyorum.,nonidiom,1.00,5,0,0,"[2, 3]","['üzerinde', 'duramıyorum']","['bu', 'ip', 'Üzer', 'durmak', '.']","['Bu', 'ipin', 'üzerinde', 'duramıyorum', '.']",1
2080,üzerinde durmak,İpin üzerinde tek ayağıyla duruyordu.,nonidiom,1.00,5,0,0,"[1, 4]","['üzerinde', 'duruyordu']","['ip', 'Üzer', 'tek', 'Ayağ', 'durmak', '.']","['İpin', 'üzerinde', 'tek', 'ayağıyla', 'duruy...",1
2081,üzerinde durmak,Bilgisayarın üzerinde kitapları duruyor,nonidiom,1.00,4,0,0,"[1, 3]","['üzerinde', 'duruyor']","['bilgisayar', 'Üzer', 'kitap', 'durmak']","['Bilgisayarın', 'üzerinde', 'kitapları', 'dur...",1
2082,üzerinde durmak,Üzerinde çilekler duran tartı bana uzattı.,nonidiom,1.00,1,0,0,"[0, 2]","['Üzerinde', 'duran']","['Üzer', 'Çilek', 'durmak', 'tartı', 'ban', 'u...","['Üzerinde', 'çilekler', 'duran', 'tartı', 'ba...",1


In [44]:
class TextDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)


In [45]:
# Initialize tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

# Tokenize the data
train_encodings = tokenizer(list(df_train['submission']), truncation=True, padding=True, return_tensors="pt")
val_encodings = tokenizer(list(df_eval['submission']), truncation=True, padding=True, return_tensors="pt")
test_encodings = tokenizer(list(df_test['submission']), truncation=True, padding=True, return_tensors="pt")
# Convert to torch tensors
train_labels = torch.tensor(df_train['category_encoded'].values)
val_labels = torch.tensor(df_eval['category_encoded'].values)
test_labels = torch.tensor(df_test['category_encoded'].values)

# Prepare to datasets
train_dataset = TextDataset(train_encodings, train_labels)
val_dataset = TextDataset(val_encodings, val_labels)
test_dataset = TextDataset(test_encodings, test_labels)

# DataLoader
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=8, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False)

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [ ]:
num_labels = len(label_encoder.classes_)
model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased', num_labels=num_labels)

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)
optimizer = AdamW(model.parameters(), lr=5e-6)

epochs = 4
for epoch in range(epochs):
    model.train()
    total_loss = 0
    for batch in tqdm(train_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.item()

        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

    # Average loss across all batches
    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs} | Train Loss: {avg_train_loss}")

    # Evaluation step
    model.eval()
    total_eval_accuracy = 0
    for batch in tqdm(val_loader):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)

        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)
        accuracy = (predictions == batch['labels']).cpu().numpy().mean() * 100
        total_eval_accuracy += accuracy

    avg_val_accuracy = total_eval_accuracy / len(val_loader)
    print(f"Epoch {epoch + 1}/{epochs} | Validation Accuracy: {avg_val_accuracy:.2f}%")


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
  0%|          | 0/288 [00:00<?, ?it/s]<ipython-input-44-845b27163e62>:7: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
<ipython-input-44-845b27163e62>:8: User

Epoch 1/4 | Train Loss: 0.3597753508430388


100%|██████████| 72/72 [00:02<00:00, 32.97it/s]


Epoch 1/4 | Validation Accuracy: 90.62%


 93%|█████████▎| 269/288 [00:43<00:03,  6.12it/s]

In [ ]:
# Ensure the model is in evaluation mode
model.eval()

# Move the model to the appropriate device
model.to(device)

# Initialize variables to track accuracy
total_correct = 0
total_predictions = 0

# Store true labels and predictions for F1 score calculation
all_labels = []
all_predictions = []

# No gradient updates needed for evaluation
with torch.no_grad():
    for batch in test_loader:
        # Move batch to the same device as the model
        batch = {k: v.to(device) for k, v in batch.items()}

        # Forward pass
        outputs = model(**batch)

        # Get predictions
        logits = outputs.logits
        predictions = torch.argmax(logits, dim=-1)

        # Update tracking variables
        total_correct += (predictions == batch['labels']).sum().item()
        total_predictions += predictions.size(0)

        # Store predictions and true labels
        all_predictions.extend(predictions.cpu().numpy())
        all_labels.extend(batch['labels'].cpu().numpy())

# Calculate accuracy
test_accuracy = total_correct / total_predictions
print(f'Test Accuracy: {test_accuracy:.4f}')

# Calculate F1 scores
f1_weighted = f1_score(all_labels, all_predictions, average='weighted')
f1_macro = f1_score(all_labels, all_predictions, average='macro')

print(f'Weighted F1 Score: {f1_weighted:.4f}')
print(f'Macro F1 Score: {f1_macro:.4f}')


In [ ]:
# Generate classification report
report = classification_report(all_labels, all_predictions, digits=4)
print(f'Classification Report for {modelname}, language {language.upper()}, Train {input.upper()}, Test {test.upper()}:')
print(report)